<a href="https://colab.research.google.com/github/RogerJL/LTU/blob/main/eMaintenance/machine_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Submit your solutions in pdf format, with code and plots supporting your answers.
machine_data contains raw data of a part from 3 manufactures A, B, C
The system is run to failure under load
The load and the operation time is provided in each row

What is the range of load and time during operation for each manufacturer?
What is the most expected load value?
How are the load and time related?
Which distribution best describes the load?
Which distribution best describes the time?

Which manufacturer has the best performance and why?


In [23]:

import numpy as np
import pandas as pd

import matplotlib
import lets_plot as lplt
import matplotlib.pyplot as plt
#matplotlib.use("TkAgg")

%matplotlib notebook

In [11]:
# read the data file into a dataframe
df = pd.read_csv('machine_data.csv')
print(df)

print(df.shape)


      Unnamed: 0       time       load manufacturef
0              0  20.008150  70.446291            c
1              1  14.014561  73.294634            c
2              2  26.701731  73.840790            A
3              3  14.809997  77.084801            B
4              4  10.626743  75.508418            c
...          ...        ...        ...          ...
1224        1224  11.964054  74.560155            c
1225        1225   9.787055  76.166923            c
1226        1226  22.561823  75.862420            A
1227        1227  25.499175  74.393778            A
1228        1228  13.919766  77.704726            B

[1229 rows x 4 columns]
(1229, 4)


Drop the index

In [12]:
df.drop(columns='Unnamed: 0', inplace=True)
minimum = df.min()
maximum = df.max()


Extract data for a given manufacturer

In [31]:
lplt.LetsPlot.setup_html()

np.random.seed(12)
data = dict(
    cond=np.repeat(['A', 'B'], 200),
    rating=np.concatenate((np.random.normal(0, 1, 200), np.random.normal(1, 1.5, 200)))
)

#lplt.ggplot(data, lplt.aes(x='rating', fill='cond')) + lplt.ggsize(700, 300) + \
#    lplt.geom_density(color='dark_green', alpha=.7) + lplt.scale_fill_brewer(type='seq') + \
#    lplt.theme(panel_grid_major_x='blank')

lplt.ggplot(df, lplt.aes(x='load', y='time')) + \
    lplt.ggsize(700, 300) + \
    lplt.geom_density(color='dark_green', alpha=.7) + \
    lplt.scale_fill_brewer(type='seq') + \
    lplt.theme(panel_grid_major_x='blank'))
#lplt.show()

In [13]:

grpByManu = df.groupby(['manufacturef'])

fig, axs = plt.subplots(3 + 1, 1)
ax, axs = axs[0], axs[1:]
ax.axis([minimum['load'], maximum['load'], minimum['time'], maximum['time']])
ax.set_title("Relation between load and time")
ax.set_xlabel("Load")
ax.set_ylabel("Time")

for index, (name, dfa) in enumerate(grpByManu):
    name = "Manufacturer " + name[0]

    load = dfa['load']
    time = dfa['time']

    #%%
    '''
    Is there a relationship between load and time
    '''
    path_collection = ax.scatter(load, time)
    path_collection.set_label(name)

    #%%
    '''
    Characteristics of data
    mean, median, mode
    '''
    print(f"{name}, load mean={dfa['load'].mean()}, median={dfa['load'].median()}, mode={dfa['load'].mode()}")
    #%%
    '''
    How is load distributed
    Why does it matter
    uniform, normal, exponential, weibull
    '''
    x = dfa[['load']].plot(ax=axs[index], kind='hist', bins=10)
    x.set_label(name)
    x.set_title(f"Histogram of load distribution")

    #%%
    '''
    variance, standard deviation
    What is the meaning of 6sigma
    '''
    print(f"{name}, load var={dfa['load'].var()}, stddev={dfa['load'].std()}")
    #%%
    '''
    Other plots that can be useful 
    boxplot
    '''
ax.legend()
plt.show()


AttributeError: module 'lets_plot' has no attribute 'show'